In [17]:
import re
from collections import defaultdict

In [32]:
def get_table_names(input_file, output_file):
    pattern = re.compile(
        r"INSERT INTO\s+(\[[^\]]+\]\.\[[^\]]+\])",
        re.IGNORECASE)
    table_names = []
    try:
        with open(input_file, 'r', encoding='utf-8') as input_file:
            # Sử dụng enumerate(..., 1) để lấy cả số dòng (bắt đầu từ 1) và nội dung
            for line in input_file:
                 # Sử dụng re.search() để tìm kiếm mẫu trong từng dòng
                match = pattern.search(line)
                
                # Nếu tìm thấy mẫu
                if match:
                    # match.group(0) sẽ là toàn bộ chuỗi khớp (vd: 'INSERT INTO  [dbo].[task_files]')
                    # match.group(1) sẽ là chuỗi trong cặp ngoặc đơn đầu tiên của pattern
                    table_name = match.group(1)
                    table_names.append(table_name)
                    print(f"  - Tìm thấy: {table_name} trong dòng: \"{line.strip()}\"")


        # Ghi kết quả thống kê này ra file
        unique_table_names = sorted(list(set(table_names)))
        with open(output_file, 'w', encoding='utf-8') as f_out:
            if unique_table_names:
                f_out.write(f"Thống kê {len(unique_table_names)} bảng được tìm thấy trong file '{input_sql_file}':\n\n")
                
                # Sắp xếp theo tên bảng để kết quả có thứ tự
                for table_name in sorted(unique_table_names):                    
                    f_out.write(f"{table_name};\n")
                    
            else:
                f_out.write(f"Không tìm thấy bảng nào.\n")

        print(f"Đã ghi thống kê chi tiết vào file '{output_file}'")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file nguồn '{input_sql_file}'")
    except Exception as e:
        print(f"Đã xảy ra một lỗi không mong muốn: {e}")

In [21]:
def get_table_name_from_src(input_sql_file, output_insert_file):
    # Biểu thức chính quy để tìm chuỗi có dạng [schema].[table] theo sau "INSERT INTO"
    # re.IGNORECASE để không phân biệt chữ hoa/thường (bắt được cả INSERT INTO và insert into)
    # \s+ khớp với một hoặc nhiều khoảng trắng
    # (\[[^\]]+\]\.\[[^\]]+\]) là nhóm chúng ta muốn lấy (capturing group)
    pattern = re.compile(
        r"^Dòng\s+(\d+):\s*.*INSERT INTO\s+(\[[^\]]+\]\.\[[^\]]+\])",
        re.IGNORECASE)
    table_names = []
    try:
        with open(input_sql_file, 'r', encoding='utf-8') as input_sql_file:
            # Sử dụng enumerate(..., 1) để lấy cả số dòng (bắt đầu từ 1) và nội dung
            for line in input_sql_file:
                
                # Sử dụng re.search() để tìm kiếm mẫu trong từng dòng
                match = pattern.search(line)
                
                # Nếu tìm thấy mẫu
                if match:
                    # Nhóm 1 là số dòng
                    line_number = match.group(1)
                    # Nhóm 2 là tên bảng
                    table_name = match.group(2)
                    
                    # Thêm cặp kết quả vào danh sách
                    table_names.append((line_number, table_name))
                    print(f"  - OK: Dòng '{line_number}' -> Bảng '{table_name}'")

        # Giả sử 'results' là danh sách các cặp (số dòng, tên bảng) đã tìm được
        # Ví dụ: results = [('264994', '[dbo].[task_assignees]'), ('265996', '[dbo].[task_assignees]'), ('266998', '[dbo].[tasks]')]

        # Tạo một dictionary để nhóm các dòng theo tên bảng
        tables_with_lines = defaultdict(list)
        for line_number, table_name in table_names:
            tables_with_lines[table_name].append(line_number)

        # Kết quả của 'tables_with_lines' sẽ là:
        # {
        #   '[dbo].[task_assignees]': ['264994', '265996'],
        #   '[dbo].[tasks]': ['266998']
        # }

        # Ghi kết quả thống kê này ra file
        with open(output_insert_file, 'w', encoding='utf-8') as f_out:
            if tables_with_lines:
                f_out.write(f"Thống kê {len(tables_with_lines)} bảng được tìm thấy trong file '{input_sql_file}':\n\n")
                
                # Sắp xếp theo tên bảng để kết quả có thứ tự
                for table_name in sorted(tables_with_lines.keys()):
                    line_numbers = tables_with_lines[table_name]
                    # Nối các số dòng thành một chuỗi, cách nhau bởi dấu phẩy
                    lines_str = ', '.join(line_numbers)
                    
                    f_out.write(f"- Bảng: {table_name}\n")
                    f_out.write(f"  -> Xuất hiện ở các dòng: {lines_str}\n\n")
                    
            else:
                f_out.write(f"Không tìm thấy bảng nào.\n")

        print(f"Đã ghi thống kê chi tiết vào file '{output_insert_file}'")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file nguồn '{input_sql_file}'")
    except Exception as e:
        print(f"Đã xảy ra một lỗi không mong muốn: {e}")

In [35]:
input_sql_file = 'insert_lines_of_PronaFlow_New_Sample_Data_EN.txt'

In [34]:
output_insert_file = 'table_names.txt'
get_table_name_from_src(input_sql_file, output_insert_file)

  - OK: Dòng '46' -> Bảng '[dbo].[users]'
  - OK: Dòng '88' -> Bảng '[dbo].[workspaces]'
  - OK: Dòng '338' -> Bảng '[dbo].[tags]'
  - OK: Dòng '1340' -> Bảng '[dbo].[tags]'
  - OK: Dòng '1848' -> Bảng '[dbo].[projects]'
  - OK: Dòng '2850' -> Bảng '[dbo].[projects]'
  - OK: Dòng '3852' -> Bảng '[dbo].[projects]'
  - OK: Dòng '4854' -> Bảng '[dbo].[projects]'
  - OK: Dòng '5677' -> Bảng '[dbo].[project_members]'
  - OK: Dòng '6679' -> Bảng '[dbo].[project_members]'
  - OK: Dòng '7681' -> Bảng '[dbo].[project_members]'
  - OK: Dòng '8683' -> Bảng '[dbo].[project_members]'
  - OK: Dòng '9685' -> Bảng '[dbo].[project_members]'
  - OK: Dòng '10687' -> Bảng '[dbo].[project_members]'
  - OK: Dòng '11689' -> Bảng '[dbo].[project_members]'
  - OK: Dòng '12691' -> Bảng '[dbo].[project_members]'
  - OK: Dòng '13693' -> Bảng '[dbo].[project_members]'
  - OK: Dòng '14695' -> Bảng '[dbo].[project_members]'
  - OK: Dòng '15697' -> Bảng '[dbo].[project_members]'
  - OK: Dòng '16699' -> Bảng '[dbo].[p

In [33]:
output_file = 'table_names_simple.txt'
get_table_names(input_sql_file, output_file)

  - Tìm thấy: [dbo].[users] trong dòng: "Dòng 46: INSERT INTO [dbo].[users] ([username], [email], [password_hash], [full_name], [bio], [role]) VALUES"
  - Tìm thấy: [dbo].[workspaces] trong dòng: "Dòng 88: INSERT INTO [dbo].[workspaces] ([owner_id], [name], [description]) VALUES"
  - Tìm thấy: [dbo].[tags] trong dòng: "Dòng 338: INSERT INTO [dbo].[tags] ([workspace_id], [name], [color_hex]) VALUES"
  - Tìm thấy: [dbo].[tags] trong dòng: "Dòng 1340: INSERT INTO [dbo].[tags] ([workspace_id], [name], [color_hex]) VALUES"
  - Tìm thấy: [dbo].[projects] trong dòng: "Dòng 1848: INSERT INTO [dbo].[projects] ([workspace_id], [name], [description], [status], [project_type], [start_date], [end_date], [is_archived]) VALUES"
  - Tìm thấy: [dbo].[projects] trong dòng: "Dòng 2850: INSERT INTO [dbo].[projects] ([workspace_id], [name], [description], [status], [project_type], [start_date], [end_date], [is_archived]) VALUES"
  - Tìm thấy: [dbo].[projects] trong dòng: "Dòng 3852: INSERT INTO [dbo].[proj